In [16]:
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision import transforms
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import numpy as np
import torch
import json

In [17]:
def one_hot_labels(labels):
    r = np.zeros((labels.size, labels.max()+1))
    r[np.arange(labels.size),labels] = 1
    return r

In [19]:
import LBP.LBP_classes as LBP_classes
path="Best_Models/LBPBoth/weights-v0.ckpt"
model = LBP_classes.LBPBoth.load_from_checkpoint(path)

In [20]:
# Get test dataloader
transform = transforms.Compose([
        transforms.Grayscale(1),
        transforms.ToTensor()
    ])
test_data = CIFAR10(root="data/CIFAR10", download=False, train=False, transform=transform)
test_loader = DataLoader(test_data, batch_size=32, num_workers=12)
print ("DataLoader acquired")

DataLoader acquired


In [21]:
# Prediction
y_true = torch.cat([labels for _, labels in test_loader]).reshape(-1).numpy()
y_roc = one_hot_labels(y_true)
y_pred = torch.cat([model(images) for images, _ in test_loader]).detach().numpy()
print ("Predictions calculated")

Predictions calculated


In [22]:
auc = roc_auc_score(y_roc, y_pred, multi_class="ovr")
print("Model AUC:", auc)

Model AUC: 0.8752679222222222


In [23]:
model_acc = accuracy_score(y_true, y_pred.argmax(-1))
print("Model accuracy:", model_acc)

Model accuracy: 0.6335
